# UTSA CS 3793/5233: Assignment-2

Fall 2021


**Murphey - Quinn - (pfl955)**






## Learning Objectives



*   Game Playing
*   Chess - Board Setup & Rules
*   Adversarial Search
*   AI - Random vs MinMax



## Description

This assignment is focused on **game playing** and creating a proper **AI for chess**. 
In the following sections, you will complete a series of tasks to create a chess game board, rules for each chess piece, a Random AI and a MinMax AI that plays a game of chess for both players (white and black).

The base structure of the code is provided. You are supposed to write code for each of the functions. Comments are provided on what should be done. You **CANNOT** use a complete chess library and change the base code structure completely. However, you **CAN** change the code layout and name/format of the functions.


#Chess Board Setup & Rules

In this section, you will write code to import the necessary libraries and create:

1.   **ChessBoard** - This part will contain code to initialize the board, draw the board, get the board state and move piece.
2.   **ChessRules** - This part will contain code for the chess rules for each piece.



##Import Libraries

The code here will contain only **import** statements. A base list of the required libraries are already imported. You will most likely not need any other libraries, but if needed, add the import statements here. As mentioned before, you can not use any premade chess libraries.

In [1]:
# your code goes here
import string
import random
import os
import sys
import time
import math
from IPython.display import clear_output
import numpy as np


## ChessBoard

(10 points)

Fill the code in the code structure provided below for the ChessBoard. The main use of this code block write functions to initialize the board, draw the board, get the board state and move piece. You can add any other functions if needed.


## ChessRules

(60 points)

Fill the code in the code structure provided below for ChessRules. The main use of the code block is to write functions to design the rules for movement of each piece on the board. This block will also contain the function to check if the current player is in check, check-mate. You can also have functions that can return the current player's pieces that have legal moves in the current board state. 

Following are some **suggested** functions with the pseudocode provided. You can create/remove functions as needed.


In [2]:
# you can add/change the input parameters for each function 
# you can change the function names and also add more functions if needed


def ChessBoardSetup():
    # initialize and return a chess board - create a 2D 8x8 array that has the value for each cell
    board = np.full((8,8),'.', np.dtype('U1')) #creates an 8x8 array of single unicode characters filled with '.'
    # USE the following characters for the chess pieces - lower-case for BLACK and upper-case for WHITE
    # . for empty board cell
    # p/P for pawn
    board[6,:] = 'P'
    board[1,:] = 'p'
    # r/R for rook
    board[7,[0,7]] = 'R'
    board[0,[0,7]] = 'r'
    # t/T for knight
    board[7,[1,6]] = 'T'
    board[0,[1,6]] = 't'
    # b/B for bishop
    board[7,[2,5]] = 'B'
    board[0,[2,5]] = 'b'
    # q/Q for queen
    board[7,3] = 'Q'
    board[0,3] = 'q'
    # k/K for king
    board[7,4] = 'K'
    board[0,4] = 'k'
    return board


def DrawBoard(board):
    # write code to print the board - following is one print example
    # r t b q k b t r
    # p p p p p p p p
    # . . . . . . . .
    # . . . . . . . .
    # . . . . . . . .
    # . . . . . . . .
    # P P P P P P P P
    # R T B Q K B T R
    for row in board: 
        for col in row:
            print(col + ' ', end='') # print the character of the piece with a space after it and no endline
        print()


def MovePiece(board,from_coord,to_coord):
    """
    Moves a piece from from_coord to to_coord on the board. 
    Does not check move validity, and overwrites any piece on to_coord.
        
    Params:
        board: 8x8 numpy array with dtype = "U1"
        from_coord: 2 integer tuple (row, col)
        to_coord: 2 integer tuple (row, col)
    """
    # set piece at to_coord to the piece from from_coord
    board[to_coord[0],to_coord[1]] = board[from_coord[0],from_coord[1]]
    # mark from_coord as empty
    board[from_coord[0],from_coord[1]] = '.'    

In [3]:
def IsMoveLegal(board, from_coord, to_coord):
    """
    Returns True is the proposed move from from_coord to to_coord is legal, else False
    
    Params:
        board: 8x8 numpy array with dtype = "U1"
        from_coord: 2 integer tuple (row, col)
        to_coord: 2 integer tuple (row, col)
    """
    # input is from-square and to-square
    # use the input and the board to get the from-piece and to-piece
    from_piece = board[from_coord[0],from_coord[1]]
    to_piece = board[to_coord[0], to_coord[1]]
    
    # How the piece is moving
    coord_diff = [to_coord[0] - from_coord[0], to_coord[1] - from_coord[1]]
    
    # True iff from_piece is white
    white = from_piece.isupper() 
    
    # if not a move
    if from_coord == to_coord:
        return False
    
    # if from_piece is a pawn
    if from_piece.lower() == 'p': 
        # if move one step forward and to_coord is empty
        if (
            to_piece == '.'
            and (
                (white and coord_diff == [-1,0]) 
                or (not white and coord_diff == [1,0])
            )
        ):
            return True
        # if move two steps forward, to_coord is empty, from_piece is on starting row, and IsClearPath()
        if (
            to_piece == '.'
            and IsClearPath(board, from_coord, to_coord)
            and (
                (white and coord_diff == [-2,0] and from_coord[0] == 6)
                or ((not white) and coord_diff == [2,0] and from_coord[0] == 1)
            )
        ):
            return True
        
        # if move diagonally forwards one and to_coord has an enemy on it
        if (
            ( 
                white
                and (
                    coord_diff == [-1,-1]
                    or coord_diff == [-1,1]
                )
                and to_piece.islower()
            )
            or (
                not white
                and (
                    coord_diff == [1,-1]
                    or coord_diff == [1,1]
                )
                and to_piece.isupper()
            )
        ):
            return True

    # if from_piece is a rook
    if from_piece.lower() == 'r':
        # if piece either moving horizontally or vertically, to_piece either empty or enemy,
        # and the path is clear
        if (
            0 in coord_diff
            and (
                to_piece == '.'
                or (white and to_piece.islower())
                or (not white and to_piece.isupper())
            )
            and IsClearPath(board, from_coord, to_coord)
        ):
            return True
    
    # if from_piece is a bishop
    if from_piece.lower() == 'b':
        # if piece moving diagonally, to_piece either empty or enemy, and the path is clear
        if (
            abs(coord_diff[0]) == abs(coord_diff[1])
            and (
                to_piece == '.'
                or (white and to_piece.islower())
                or (not white and to_piece.isupper())
            )
            and IsClearPath(board, from_coord, to_coord)
        ):
            return True
    
    if from_piece.lower() == 'q':
        # if piece either moving horizontally or vertically, to_piece either empty or enemy,
        # and the path is clear
        if (
            0 in coord_diff
            and (
                to_piece == '.'
                or (white and to_piece.islower())
                or (not white and to_piece.isupper())
            )
            and IsClearPath(board, from_coord, to_coord)
        ):
            return True
        
        # if piece moving diagonally, to_piece either empty or enemy, and the path is clear
        if (
            abs(coord_diff[0]) == abs(coord_diff[1])
            and (
                to_piece == '.'
                or (white and to_piece.islower())
                or (not white and to_piece.isupper())
            )
            and IsClearPath(board, from_coord, to_coord)
        ):
            return True

    # if from_piece is a knight
    if from_piece.lower() == 't':
        # if to_piece is empty or enemy, and piece moves as a knight moves
        if (
            (
                (coord_diff[0] == 1 and coord_diff[1] == -2)
                or (coord_diff[0] == 2 and coord_diff[1] == -1)
                or (coord_diff[0] == 2 and coord_diff[1] == 1)
                or (coord_diff[0] == 1 and coord_diff[1] == 2)
                or (coord_diff[0] == -1 and coord_diff[1] == -2)
                or (coord_diff[0] == -2 and coord_diff[1] == -1)
                or (coord_diff[0] == -2 and coord_diff[1] == 1)
                or (coord_diff[0] == -1 and coord_diff[1] == 2)
            )
            and (
                to_piece == '.'
                or (white and to_piece.islower())
                or (not white and to_piece.isupper())
            )
        ):
            return True
    
    # if from_piece is a king
    if from_piece.lower() == 'k':
        # if to_piece is empty or enemy, and piece moves 1 space orthogonally or diagonally
        if (
            coord_diff[0] in [-1,0,1]
            and coord_diff[1] in [-1,0,1]
            and (
                to_piece == '.'
                or (white and to_piece.islower())
                or (not white and to_piece.isupper())
            ) 
        ):
            return True

    return False 

def GetListOfLegalMoves(board, from_coord):
    """
    Returns a list of legal moves for the piece at from_coord
    
    Params:
        board: 8x8 numpy array with dtype = "U1"
        from_coord: 2 integer tuple (row, col)
    """
    
    legalMoves = []
    # loop over all squares on the board
    for row in range(8):
        for col in range(8):
            # if from_coord can move to (row, col) and doesn't put the current player in check
            if (
                IsMoveLegal(board, from_coord, (row, col))
                and not DoesMovePutPlayerInCheck(board, from_coord, (row, col))
            ):
                # add move to legalMoves
                legalMoves.append((row, col)) 
    # return the list of legal moves
    return legalMoves

def GetPiecesWithLegalMoves(board, player):
    """
    Returns a list of all pieces with legal moves for current player
    
    Params:
        board: 8x8 numpy array with dtype = "U1"
        players: either 'b' or 'w' representing black or white respectively
    """
    
    # initialize the list of pieces with legal moves to []
    piecesWithLegalMoves = []
    # loop over all squares on the board
    for row in range(8):
        for col in range(8):
            # if piece belongs to player and has a legal move
            if (
                (
                    (player == 'w' and board[row,col].isupper())
                    or (player == 'b' and board[row,col].islower())
                )
                and GetListOfLegalMoves(board, (row,col))
            ):
                # append this piece to the list of pieces with legal moves
                piecesWithLegalMoves.append((row,col))
    
    # return the final list of pieces with legal moves
    return piecesWithLegalMoves

def IsCheckmate(board, player):
    """
    Returns True if the current player is in checkmate, else False
    
    Params:
        board: 8x8 numpy array with dtype = "U1"
        players: either 'b' or 'w' representing black or white respectively
    """
    # if there are legal moves for the current player return False
    if GetPiecesWithLegalMoves(board, player):
        return False
    # Else return True
    return True

# returns True if the given player is in Check state
def IsInCheck(board, player):
    """
    Return True if the given player is in Check
    
    Params:
        board: 8x8 numpy array with dtype = "U1"
        players: either 'b' or 'w' representing black or white respectively
    """
    
    king_coord = None
    # loop over all squares
    for row in range(8):
        for col in range(8):
            # if (row, col) contains the players King
            if(
                (player == 'w' and board[row,col] == 'K')
                or (player == 'b' and board[row,col] == 'k')
            ):
                # set king_coord and break out of both for loops
                king_coord = (row,col)
                break
        if king_coord:
            break
    
    # loop over all squares
    for row in range(8):
        for col in range(8):
            # if enemy piece and move from (row, col) to king_coord is legal
            if(
                (
                    (player == 'w' and board[row,col].islower())
                    or (player == 'b' and board[row,col].isupper())
                )
                and IsMoveLegal(board, (row,col), king_coord)
            ):
                return True
    
    # no legal move to the player's king was found
    return False

# helper function to figure out if a move is legal for straight-line moves (rooks, bishops, queens, pawns)
# returns True if the path is clear for a move (from-square and to-square), non-inclusive
def IsClearPath(board, from_coord, to_coord):
    """
    Returns True if the path is clear for a move from from_coord to to_coord, non-inclusive
    
    Params:
        board: 8x8 numpy array with dtype = "U1"
        from_coord: 2 integer tuple (row, col)
        to_coord: 2 integer tuple (row, col)
    """
    coord_diff = [to_coord[0] - from_coord[0], to_coord[1] - from_coord[1]]
    # if move is only one square
    if (
        coord_diff[0] in [-1,0,1]
        and coord_diff[1] in [-1,0,1]
    ):
        return True
    else:
        # for each orthogonal or diagonal direction, if coord_diff is strictly in that direction,
        # set new_from_coord to the next coord in that direction
        if coord_diff[0] > 0 and coord_diff[1] == 0:# positive vertical
            new_from_coord = (from_coord[0] + 1, from_coord[1]) 
        elif coord_diff[0] < 0 and coord_diff[1] == 0: # negative vertical
            new_from_coord = (from_coord[0] - 1, from_coord[1]) 
        elif coord_diff[0] == 0 and coord_diff[1] > 0: # positive horizontal
            new_from_coord = (from_coord[0], from_coord[1] + 1) 
        elif coord_diff[0] == 0 and coord_diff[1] < 0: # negative horizontal
            new_from_coord = (from_coord[0], from_coord[1] - 1)
        elif (
            coord_diff[0] > 0
            and coord_diff[1] > 0
            and abs(coord_diff[0]) == abs(coord_diff[1])
        ): # NE
            new_from_coord = (from_coord[0] + 1, from_coord[1] + 1)
        elif (
            coord_diff[0] < 0
            and coord_diff[1] > 0
            and abs(coord_diff[0]) == abs(coord_diff[1])
        ): # SE
            new_from_coord = (from_coord[0] - 1, from_coord[1] + 1)
        elif (
            coord_diff[0] < 0
            and coord_diff[1] < 0
            and abs(coord_diff[0]) == abs(coord_diff[1])
        ): # SW
            new_from_coord = (from_coord[0] - 1, from_coord[1] - 1)
        elif (
            coord_diff[0] > 0
            and coord_diff[1] < 0
            and abs(coord_diff[0]) == abs(coord_diff[1])
        ): # NW
            new_from_coord = (from_coord[0] + 1, from_coord[1] - 1)
        else:
            new_from_coord = -1

    # if new_from_coord is not empty
    if new_from_coord == -1 or board[new_from_coord[0], new_from_coord[1]] != '.':
        return False
    else:
        # return the result from the recursive call of IsClearPath() with the new-from-coord and to-square
        return IsClearPath(board, new_from_coord, to_coord)

# makes a hypothetical move (from-square and to-square)
# returns True if it puts current player into check
def DoesMovePutPlayerInCheck(board, from_coord, to_coord):
    """
    Returns True if the move from from_coord to to_coord puts the player whose piece is moving in check
    
    Params:
        board: 8x8 numpy array with dtype = "U1"
        from_coord: 2 integer tuple (row, col)
        to_coord: 2 integer tuple (row, col)
    """
    # given the move (from-square and to-square), find the 'from-piece' and 'to-piece'
    from_piece = board[from_coord[0], from_coord[1]]
    to_piece = board[to_coord[0], to_coord[1]]

    # make the move temporarily by changing the 'board'
    MovePiece(board, from_coord, to_coord)
    
    # Call the IsInCheck() function to see if the 'player' is in check - save the returned value
    if from_piece.isupper():
        inCheck = IsInCheck(board,'w')
    else:
        inCheck = IsInCheck(board,'b')

    # Undo the temporary move
    board[from_coord[0], from_coord[1]] = from_piece
    board[to_coord[0], to_coord[1]] = to_piece
    
    # return the value saved - True if it puts current player into check, False otherwise
    return inCheck

#Artificial Intelligence

In this section, you will write code for the Artificial Intelligence (AI) that will play a game of chess. You will write 2 types of AI:

1.   **RandomAI** - This part will contain code for moving a chess piece randomly.
2.   **MinMaxAI** - This part will contain code for moving a chess piece using the MinMax strategy discussed in the lecture.


##RandomAI

(10 points)

Complete the function below that will perform a random move for the given player. The function will return the move (from-piece and to-piece). You will most likely not need to write any other function, but you can, if needed.


In [4]:
def GetRandomMove(board, player):
    """
    Returns a legal from_coord and to_coord randomly for the given player
    
    Params:
        board: 8x8 numpy array with dtype = "U1"
        player: either 'b' or 'w' representing black or white respectively
    
    Returns:
        from_coord: 2 integer tuple (row, col)
        to_coord: 2 integer tuple (row, col)
    """
    if not GetPiecesWithLegalMoves(board, player):
        return None, None
    from_coord = random.choice(GetPiecesWithLegalMoves(board, player))
    to_coord = random.choice(GetListOfLegalMoves(board, from_coord))
    return from_coord, to_coord

##MinMaxAI

(60 points)

Complete the functions below that will perform a move for the given player using the MinMax AI strategy. One function will evaluate the board if a move is performed - give score for each of piece and calculate the score for the entire chess board. In the second function you will write actual code for the MinMax strategy and return the move (from-piece and to-piece). To get the allocated points, searching should be **2-ply (one Max and one Min)**. You will most likely not need to write any other function, but you can, if needed.

## Extra Credit

*   **(5 points)** Modify the above MinMax strategy to be **4-ply (one Max, one Min, one Max, one Min)**.
*   **(15 points)** Perform **alpha-beta pruning** for the MinMax strategy.

In [7]:
def evl(board):
    """
    Returns the score on the board
    Score is calculated by calculating white's score and subtracting black's score
    Each color's score is determined by the sum of the value of its pieces where
        Pawn = 1
        Knight / Bishop = 3
        Rook = 5
        Queen = 9
    Unless the move would checkmate them, then their score would be -inf.
    
    Params:
        board: 8x8 numpy array with dtype = "U1"
        player: either 'b' or 'w' representing black or white respectively
        from_coord: 2 integer tuple (row, col)
    """
    # define point dict
    points = {
        'k' : 0,
        'p' : 1,
        'b' : 3,
        't' : 3,
        'r' : 5,
        'q' : 9
    }
    
    # save former state
    #from_piece = board[from_coord[0], from_coord[1]]
    #to_piece = board[to_coord[0], to_coord[1]]
    
    # make move
    #board[to_coord[0], to_coord[1]] = board[from_coord[0], from_coord[1]]
    
    white_score = 0
    black_score = 0
    
    # if either player is in checkmate return math.inf as their opponent's score
    if IsCheckmate(board, 'w'):
        black_score = math.inf
    elif IsCheckmate(board, 'b'):
        white_score = math.inf
    else: # else add up score of each side
        # loop over all pieces
        for row in range(8):
            for col in range(8):
                # add score of piece to respective score
                if board[row,col].isupper():
                    white_score += points[board[row,col].lower()]
                elif board[row,col].islower():
                    black_score += points[board[row,col]]
    
    # return board to original state
    #board[from_coord[0], from_coord[1]] = from_piece
    #board[to_coord[0], to_coord[1]] = to_piece

    # return overall evaluation of the board
    return white_score - black_score

def GetBestMove(board, player):
    # initialize bestScore
    if player == 'w':
        best_score = -math.inf
    else:
        best_score = math.inf
        
    best_coords = []
    
    if not GetPiecesWithLegalMoves(board, player):
        if depth == 0:
            return None, None
        else:
            return 0
    
    # loop over all legal moves
    for from_coord in GetPiecesWithLegalMoves(board, player):
        for to_coord in GetListOfLegalMoves(board, from_coord):
            from_piece = board[from_coord[0], from_coord[1]]
            to_piece = board[to_coord[0], to_coord[1]]
            # perform legal move
            MovePiece(board, from_coord, to_coord)
            
            value = evl(board)
            
            if(
                (player == 'w' and value > best_score)
                or (player == 'b' and value < best_score)
            ):
                best_score = value
                best_coords = []
                best_coords.append((from_coord,to_coord))
            elif value == best_score:
                best_coords.append((from_coord, to_coord))
                
            # return board state
            board[from_coord[0], from_coord[1]] = from_piece
            board[to_coord[0], to_coord[1]] = to_piece
            
    if best_coords:
        return random.choice(best_coords)
    else:
        return None, None
            
            

def GetMinMaxMove(board, player,depth=0,alpha=-math.inf,beta=math.inf):
    """
    Return the best move for the current player using the 4-ply MinMax strategy
    to evaluate
    
    Params:
        board: 8x8 numpy array with dtype = "U1"
        player: either 'b' or 'w' representing black or white respectively
    
    Returns (if depth=0):
        from_coord: 2 integer tuple (row, col)
        to_coord: 2 integer tuple (row, col)
        
    Returns (if depth!=0):
        score: int
    """
    
    # initialize bestScore
    if player == 'w':
        best_score = -math.inf
    else:
        best_score = math.inf
        
    
    best_from_coord = None
    best_to_coord = None
    
    # determine nextPlayer
    if player == 'w':
        next_player = 'b'
    else:
        next_player = 'w'
    
    if not GetPiecesWithLegalMoves(board, player):
        if depth == 0:
            return None, None
        else:
            return 0
    
    # loop over all legal moves
    for from_coord in GetPiecesWithLegalMoves(board, player):
        for to_coord in GetListOfLegalMoves(board, from_coord):
            # store former state
            from_piece = board[from_coord[0], from_coord[1]]
            to_piece = board[to_coord[0], to_coord[1]]
            # perform legal move
            MovePiece(board, from_coord, to_coord)
            
            # return evaluation if depth = 3 or checkmate has occurred
            if (
                depth == 3
                or IsCheckmate(board, next_player)
            ):
                value = evl(board)
            # calculate recursively if not a leaf node
            else:
                # calculate value of board recursively
                value = GetMinMaxMove(board, next_player, depth+1, alpha, beta)
                
            # determine if score is better for current player    
            if(
                (player == 'w' and value > best_score)
                or (player == 'b' and value < best_score)
            ):
                best_score = value
                best_from_coord = from_coord
                best_to_coord = to_coord
            
            # update alpha and beta
            if player == 'w':
                alpha = max(alpha, best_score)
            elif player == 'b':
                beta = min(beta, best_score)
                
            # return board state
            board[from_coord[0], from_coord[1]] = from_piece
            board[to_coord[0], to_coord[1]] = to_piece
            
            if beta <= alpha:
                break
        if beta <= alpha:
            break
    
    if depth == 0:
        return best_from_coord, best_to_coord
    return best_score

SyntaxError: invalid syntax (<ipython-input-7-39c31353a3bd>, line 42)

#Game Setup & Main Loop

(10 points)

Write code below to have a game-play between two AIs - Random vs MinMax. For each iteration, draw the board before and after a turn. 

In [ ]:
# initialize and setup the board
board = ChessBoardSetup()
# player assignment and counter initializations
player = 'w'
turns = 0
N = 30

# main game loop - while a player is not in checkmate or stalemate (<N turns)
# below is the rough looping strategy
while not IsCheckmate(board, player) and turns < N:
    turns += 1
    # write code to take turns and move the pieces
    if player == 'w':
        from_coord, to_coord = GetMinMaxMove(board, 'w')
        if from_coord == None:
            break
    elif player == 'b':
        from_coord, to_coord = GetRandomMove(board, 'b')
        if from_coord == None:
            break
            
    clear_output()
    print("Turn " + str(turns) + f"({evl(board)})")
    DrawBoard(board)
    print("-"*15)
    MovePiece(board, from_coord, to_coord)
    DrawBoard(board)
    if player == 'w':
        print("White played " + str(from_coord) + " to " + str(to_coord))
        player = 'b'
        time.sleep(3)
    elif player == 'b':
        print("Black played " + str(from_coord) + " to " + str(to_coord))
        player = 'w'

if from_coord == None:
    print("Stalemate!")
else:
    print("Checkmate!")
DrawBoard(board)


In [8]:
# initialize and setup the board
board = ChessBoardSetup()
# player assignment and counter initializations
player = 'w'
turns = 0
N = 30

# main game loop - while a player is not in checkmate or stalemate (<N turns)
# below is the rough looping strategy
while not IsCheckmate(board, player) and turns < N:
    turns += 1
    # write code to take turns and move the pieces
    if player == 'w':
        from_coord, to_coord = GetMinMaxMove(board, 'w')
        if from_coord == None:
            break
    elif player == 'b':
        from_coord, to_coord = GetBestMove(board, 'b')
        if from_coord == None:
            break
            
    clear_output()
    print("Turn " + str(turns) + f"({evl(board)})")
    DrawBoard(board)
    print("-"*15)
    MovePiece(board, from_coord, to_coord)
    DrawBoard(board)
    if player == 'w':
        print("White played " + str(from_coord) + " to " + str(to_coord))
        player = 'b'
        time.sleep(3)
    elif player == 'b':
        print("Black played " + str(from_coord) + " to " + str(to_coord))
        player = 'w'

if from_coord == None:
    print("Stalemate!")
else:
    print("Checkmate!")
DrawBoard(board)


Turn 2(0)
r t b q k b t r 
p p p p p p p p 
. . . . . . . . 
. . . . . . . . 
P . . . . . . . 
. . . . . . . . 
. P P P P P P P 
R T B Q K B T R 
---------------
r t b q k b t r 
p p p p p p p . 
. . . . . . . . 
. . . . . . . p 
P . . . . . . . 
. . . . . . . . 
. P P P P P P P 
R T B Q K B T R 
Black played (1, 7) to (3, 7)


KeyboardInterrupt: 

#Submission Instructions

1.   Complete all tasks above - **File MUST contain the output for ALL cells**
2.   Export this notebook as .ipynb
      (File > Download as ipynb)
3.   Upload the .ipynb file on Blackboard



##Rubric

*   (10 points) Chess Board Setup
*   (60 points) Chess Rules Setup
*   (10 points) Random AI
*   (60 points) MinMax AI (2-ply)
*   (10 points) Game Main Loop - Random vs MinMax
*   (20 points) Extra Credit - 4-ply MinMax + alpha-beta pruning



